## Segmenting and Clustering Neighborhoods (Search for places of food)

#### Finding the near by food in the city of Toronto

In [3]:
import requests
import pandas as pd
import numpy as np

#load libraries
!conda install -c conda-forge geopy --yes


from geopy.geocoders import Nominatim #converts address to lat & longi

#to display images
from IPython.display import Image
from IPython.core.display import HTML

#converts json to pd.Data frame
from pandas.io.json import json_normalize


!conda install -c conda-forge folium --yes
import folium #used to plot


Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\admis\Anaconda3

  added / updated specs:
    - folium


The following packages will be UPDATED:

  folium                                         0.5.0-py_0 --> 0.10.0-py_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done


In [10]:
client_id = 'UUWDOREKVTRDTVYU0G2YHC53JE0UY4QM33AWPW5PW0GOXF1T'
client_secret = 'A3WSRIWPHWYGGSZOYKXUX1WIJDHTGBK1LPM1VCWNIJOSWVLB'
version = '20180604'
limit = 30
print('client id: ',client_id, '\n'
       'client secret: ', client_secret)

client id:  UUWDOREKVTRDTVYU0G2YHC53JE0UY4QM33AWPW5PW0GOXF1T 
client secret:  A3WSRIWPHWYGGSZOYKXUX1WIJDHTGBK1LPM1VCWNIJOSWVLB


### Define a city and gets its lat and longi

In [42]:
city = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
lati = location.latitude
longi = location.longitude
print(lati,longi)


43.653963 -79.387207


### Search for food!

In [43]:
search = 'Food'
radius = 500

#defining the URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(client_id, client_secret, lati, longi, version, search, radius, limit)
url

'https://api.foursquare.com/v2/venues/search?client_id=UUWDOREKVTRDTVYU0G2YHC53JE0UY4QM33AWPW5PW0GOXF1T&client_secret=A3WSRIWPHWYGGSZOYKXUX1WIJDHTGBK1LPM1VCWNIJOSWVLB&ll=43.653963,-79.387207&v=20180604&query=Food&radius=500&limit=30'

In [44]:
results = requests.get(url).json()


In [45]:
#results

In [47]:
venues = results['response']['venues']

#convert to dataframe
df = json_normalize(venues)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4adb87b0f964a520ad2821e3,393 Dundas St W,CA,Toronto,Canada,Beverley St,581,"[393 Dundas St W (Beverley St), Toronto ON M5T...","[{'label': 'display', 'lat': 43.65365148008927...",43.653651,-79.394413,NaN,M5T 1G6,ON,金城超級市場 Lucky Moose Food Mart,v-1568836646
1,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",False,4f147d1ce4b03f4c02b6f273,595 Bay St.,CA,Toronto,Canada,at Dundas St. W,395,"[595 Bay St. (at Dundas St. W), Toronto ON M5G...","[{'label': 'display', 'lat': 43.65616874431004...",43.656169,-79.383352,NaN,M5G 2N8,ON,Atrium On Bay Food Court,v-1568836646
2,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",False,4cbc732b035d236ac1a2e54e,81 Gerrard St W,CA,Toronto,Canada,La Plante Ave,513,"[81 Gerrard St W (La Plante Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.658497, 'lng':...",43.658497,-79.386035,NaN,M7A 2H4,ON,Saha Mediterranean Fast Food,v-1568836646
3,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",False,4c9bc2ed0e9bb1f7f8a7cd5f,140 Simcoe St,CA,Toronto,Canada,btwn Richmond St W & Nelson St,529,[140 Simcoe St (btwn Richmond St W & Nelson St...,"[{'label': 'display', 'lat': 43.64921175406505...",43.649212,-79.386916,NaN,NaN,ON,JC Food Market,v-1568836646
4,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,599754ec603d2a6715c53758,439 University Avenue,CA,Toronto,Canada,NaN,51,"[439 University Avenue, Toronto ON M5G 1Y8, Ca...","[{'label': 'display', 'lat': 43.65442, 'lng': ...",43.654420,-79.387110,NaN,M5G 1Y8,ON,Green Republic Food Co.,v-1568836646


In [49]:
#clean the data
clean_col = ['name','categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
clean_df = df.loc[:,clean_col]
clean_df.head()

,name,categories,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,id
0,金城超級市場 Lucky Moose Food Mart,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",393 Dundas St W,CA,Toronto,Canada,Beverley St,581,"[393 Dundas St W (Beverley St), Toronto ON M5T...","[{'label': 'display', 'lat': 43.65365148008927...",43.653651,-79.394413,NaN,M5T 1G6,ON,4adb87b0f964a520ad2821e3
1,Atrium On Bay Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",595 Bay St.,CA,Toronto,Canada,at Dundas St. W,395,"[595 Bay St. (at Dundas St. W), Toronto ON M5G...","[{'label': 'display', 'lat': 43.65616874431004...",43.656169,-79.383352,NaN,M5G 2N8,ON,4f147d1ce4b03f4c02b6f273
2,Saha Mediterranean Fast Food,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",81 Gerrard St W,CA,Toronto,Canada,La Plante Ave,513,"[81 Gerrard St W (La Plante Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.658497, 'lng':...",43.658497,-79.386035,NaN,M7A 2H4,ON,4cbc732b035d236ac1a2e54e
3,JC Food Market,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",140 Simcoe St,CA,Toronto,Canada,btwn Richmond St W & Nelson St,529,[140 Simcoe St (btwn Richmond St W & Nelson St...,"[{'label': 'display', 'lat': 43.64921175406505...",43.649212,-79.386916,NaN,NaN,ON,4c9bc2ed0e9bb1f7f8a7cd5f
4,Green Republic Food Co.,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",439 University Avenue,CA,Toronto,Canada,NaN,51,"[439 University Avenue, Toronto ON M5G 1Y8, Ca...","[{'label': 'display', 'lat': 43.65442, 'lng': ...",43.654420,-79.387110,NaN,M5G 1Y8,ON,599754ec603d2a6715c53758


In [55]:
# keep only columns that include venue name, and anything that is associated with location
clean_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')]+ ['id']
clean_dataframe = df.loc[:,clean_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,金城超級市場 Lucky Moose Food Mart,Grocery Store,393 Dundas St W,CA,Toronto,Canada,Beverley St,581,"[393 Dundas St W (Beverley St), Toronto ON M5T...","[{'label': 'display', 'lat': 43.65365148008927...",43.653651,-79.394413,NaN,M5T 1G6,ON,4adb87b0f964a520ad2821e3
1,Atrium On Bay Food Court,Food Court,595 Bay St.,CA,Toronto,Canada,at Dundas St. W,395,"[595 Bay St. (at Dundas St. W), Toronto ON M5G...","[{'label': 'display', 'lat': 43.65616874431004...",43.656169,-79.383352,NaN,M5G 2N8,ON,4f147d1ce4b03f4c02b6f273
2,Saha Mediterranean Fast Food,Mediterranean Restaurant,81 Gerrard St W,CA,Toronto,Canada,La Plante Ave,513,"[81 Gerrard St W (La Plante Ave), Toronto ON M...","[{'label': 'display', 'lat': 43.658497, 'lng':...",43.658497,-79.386035,NaN,M7A 2H4,ON,4cbc732b035d236ac1a2e54e
3,JC Food Market,Grocery Store,140 Simcoe St,CA,Toronto,Canada,btwn Richmond St W & Nelson St,529,[140 Simcoe St (btwn Richmond St W & Nelson St...,"[{'label': 'display', 'lat': 43.64921175406505...",43.649212,-79.386916,NaN,NaN,ON,4c9bc2ed0e9bb1f7f8a7cd5f
4,Green Republic Food Co.,Seafood Restaurant,439 University Avenue,CA,Toronto,Canada,NaN,51,"[439 University Avenue, Toronto ON M5G 1Y8, Ca...","[{'label': 'display', 'lat': 43.65442, 'lng': ...",43.654420,-79.387110,NaN,M5G 1Y8,ON,599754ec603d2a6715c53758


In [56]:
# delete unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,金城超級市場 Lucky Moose Food Mart,Grocery Store,393 Dundas St W,43.653651,-79.394413,M5T 1G6,ON
1,Atrium On Bay Food Court,Food Court,595 Bay St.,43.656169,-79.383352,M5G 2N8,ON
2,Saha Mediterranean Fast Food,Mediterranean Restaurant,81 Gerrard St W,43.658497,-79.386035,M7A 2H4,ON
3,JC Food Market,Grocery Store,140 Simcoe St,43.649212,-79.386916,NaN,ON
4,Green Republic Food Co.,Seafood Restaurant,439 University Avenue,43.654420,-79.387110,M5G 1Y8,ON
5,Shops at Sheraton Food Court,Food Court,below Queen St W,43.651569,-79.383069,NaN,ON
6,The Bowl Korean Food,Korean Restaurant,53 McCaul St,43.653371,-79.390997,M5T 2W7,ON
7,Green Box Fresh Food Bar,Sandwich Place,200 University Ave,43.650438,-79.388611,NaN,ON
8,Provo Food Bar,Tapas Restaurant,308 Dundas Street West,43.654293,-79.391788,M5T 1G4,ON
9,Tasty Chinese Food,Chinese Restaurant,Village by the Grange,43.653757,-79.390757,NaN,ON


In [57]:
# delete rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,金城超級市場 Lucky Moose Food Mart,Grocery Store,393 Dundas St W,43.653651,-79.394413,M5T 1G6,ON
1,Atrium On Bay Food Court,Food Court,595 Bay St.,43.656169,-79.383352,M5G 2N8,ON
2,Saha Mediterranean Fast Food,Mediterranean Restaurant,81 Gerrard St W,43.658497,-79.386035,M7A 2H4,ON
4,Green Republic Food Co.,Seafood Restaurant,439 University Avenue,43.654420,-79.387110,M5G 1Y8,ON
6,The Bowl Korean Food,Korean Restaurant,53 McCaul St,43.653371,-79.390997,M5T 2W7,ON
8,Provo Food Bar,Tapas Restaurant,308 Dundas Street West,43.654293,-79.391788,M5T 1G4,ON
11,New Thai Food,Asian Restaurant,109 McCaul Street,43.653646,-79.390337,M5T 3K5,ON
15,Specialty Food Shop,Food & Drink Shop,555 University Ave,43.657319,-79.387842,M5G 1X3,ON
19,Kal & Mooy East African Food,African Restaurant,238 Queen St W,43.650020,-79.390574,M5V 1Z7,ON
22,Urban Eatery,Food Court,220 Yonge St,43.655150,-79.380847,M5B 2H1,ON


In [58]:
#delete multiple entries
df_food = clean_dataframe3.drop_duplicates(subset='name', keep = 'first')
df_food

,name,categories,address,lat,lng,postalCode,state
0,金城超級市場 Lucky Moose Food Mart,Grocery Store,393 Dundas St W,43.653651,-79.394413,M5T 1G6,ON
1,Atrium On Bay Food Court,Food Court,595 Bay St.,43.656169,-79.383352,M5G 2N8,ON
2,Saha Mediterranean Fast Food,Mediterranean Restaurant,81 Gerrard St W,43.658497,-79.386035,M7A 2H4,ON
4,Green Republic Food Co.,Seafood Restaurant,439 University Avenue,43.654420,-79.387110,M5G 1Y8,ON
6,The Bowl Korean Food,Korean Restaurant,53 McCaul St,43.653371,-79.390997,M5T 2W7,ON
8,Provo Food Bar,Tapas Restaurant,308 Dundas Street West,43.654293,-79.391788,M5T 1G4,ON
11,New Thai Food,Asian Restaurant,109 McCaul Street,43.653646,-79.390337,M5T 3K5,ON
15,Specialty Food Shop,Food & Drink Shop,555 University Ave,43.657319,-79.387842,M5G 1X3,ON
19,Kal & Mooy East African Food,African Restaurant,238 Queen St W,43.650020,-79.390574,M5V 1Z7,ON
22,Urban Eatery,Food Court,220 Yonge St,43.655150,-79.380847,M5B 2H1,ON


In [60]:
#Generate map
food_map = folium.Map(location=[lati, longi], zoom_start=14)

for lat, lng, name, cate, address in zip(clean_dataframe3['lat'],\
                                         clean_dataframe3['lng'],\
                                        clean_dataframe3['name'],\
                                        clean_dataframe3['categories'],\
                                        clean_dataframe3['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat, lng],
    radius = 5,
    popup=label,
    color='green',
    fill = True,
    fill_color='green',
    fill_opacity=0.7,
    parse_html=False).add_to(food_map)
    
food_map